<img src="images/header.png" alt="Logo UCLM-ESII" align="right">

<br><br><br><br>
<h2><font color="#92002A" size=4>Trabajo Fin de Grado</font></h2>

<h1><font color="#6B001F" size=5>Generación automática de playlist de canciones <br> mediante técnicas de minería de datos</font></h1>
<h2><font color="#92002A" size=3>Parte 5 - Generación del DataSet</font></h2>

<br>
<div style="text-align: right">
    <font color="#B20033" size=3><strong>Autor</strong>: <em>Miguel Ángel Cantero Víllora</em></font><br>
    <br>
    <font color="#B20033" size=3><strong>Directores</strong>: <em>José Antonio Gámez Martín</em></font><br>
    <font color="#B20033" size=3><em>Juan Ángel Aledo Sánchez</em></font><br>
    <br>
<font color="#B20033" size=3>Grado en Ingeniería Informática</font><br>
<font color="#B20033" size=2>Escuela Superior de Ingeniería Informática | Universidad de Castilla-La Mancha</font>

</div>

---

<br>


<a id="indice"></a>
<h2><font color="#92002A" size=5>Índice</font></h2>

<br>

* [1. Introducción](#section1)
* [2. Lectura de datos](#section2)
* [3. Generación del DataSet](#section3)

<br>

---

In [1]:
# Permite establecer la anchura de la celda
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))

In [2]:
import json
import math
import os
import pandas as pd
import datetime
import zipfile

from collections import defaultdict
from tqdm.notebook import tqdm as tqdm_nb

---

<br>


<a id="section1"></a>
## <font color="#92002A">1 - Introducción</font>
<br>

Tras haber realizado una recopilación de playlists en Spotify y haber estudiado el conjunto *‘Million Playlist Dataset’*, donde hemos podido identificar y establecer diferentes criterios para filtrar los resultados de la búsqueda hasta quedarnos con con el número de muestras establecido, ya podemos generar nuestro conjunto de datos.


<br>

<div class="alert alert-block alert-info">
    
<i class="fa fa-info-circle" aria-hidden="true"></i> __Recordatorio__:
Las playlists que tenemos desde la posición 1.000.000 en adelante, se emplearán para crear el conjunto de prueba.
</div>

<div style="text-align: right">
<a href="#indice"><font size=5><i class="fa fa-arrow-circle-up" aria-hidden="true" style="color:#92002A"></i></font></a>
</div>

---

<a id="section2"></a>
## <font color="#92002A">2 - Lectura de datos</font>

<br>

En este apartado, vamos a leer el DataSet que hemos generado con la información de listas de reproducción obtenidas. Recordemos que la lista de pistas perteneciente a cada playlist no está incluida.

<br>

In [3]:
# Variables globales

# Directorio donde almacenaremos el DataSet que vamos a generar
MPD_PATH = 'MPD'

# Directorio donde se encuentran almacenadas las playlists que 
# hemos descargado
PLS_SETS_PATH = 'pls_sets'

# Directorio donde almacenaremos las listas de pistas que hemos 
# obtenido del conjunto de playlists (PLS_SETS_PATH)
TLS_SETS_PATH = 'track_lists_sets'

# Directorio empleado para guardar/leer aquellos datos generados
# que resultan de interés para diferentes tareas
DATA_PATH = 'data'

# Número de listas que almacenaremos por fichero
NUM_LISTS_FILE = 1000

In [4]:
# Ruta del fichero donde están contenidas las playlists
mpd_info_file = os.path.join(DATA_PATH, 'mpd_info_set.csv')

if os.path.isfile(mpd_info_file):
    df_mpd_info = pd.read_csv(mpd_info_file, sep=';', encoding='utf-8')
    print(f"Playlists obtenidas: {len(df_mpd_info)}")
else:
    print("¡ERROR!: Fichero 'mpd_info_set.csv' no encontrado.")

Playlists obtenidas: 1019338


<br>

Mostramos las primeras filas del DataFrame para comprobar que los datos se han cargado correctamente:

In [5]:
df_mpd_info.head()

,id,collaborative,name,num_tracks,num_followers,num_artists,num_albums,duration_ms,modified_at,num_edits,file_name
0,7u1RhzyK1ykPQ1wfemcqoR,False,Low viscosity vibes,58,3,25,38,15191903,1559722465,14,pls-set_07672.zip
1,0ZcZqlDEw4eLILfHdni8vG,False,dalanda 🐉,104,4,92,100,22361466,1558851313,75,pls-set_05170.zip
2,0Z48DgMwRPZVVTF7Y8RLB6,False,freeze pops,52,2,13,25,10643802,1552263755,7,pls-set_06675.zip
3,1G8V8nWYp6uVacS8XAPSbo,False,Golden Oldies,98,2,64,90,25418594,1556948665,34,pls-set_01145.zip


<br>

Eliminamos las listas que pertenecen al conjunto de prueba:

In [6]:
df_mpd_info = df_mpd_info[0:1000000]

<br>

Como vimos en la [libreta](00-EstudioMPD.ipynb) donde estudiamos el fragmento del *Million Playlist Dataset*, el conjunto de datos está separado en 1.000 ficheros donde cada fichero contiene 1.000 playlist. En nuestro caso, lo vamos a dividir del mismo modo.

Las playlists serán almacenadas en ficheros siguiendo el orden en el que aparecen en el DataFrame `df_mpd_info`. Para saber a qué bloque/fichero pertenece cada playlist, lo haremos de la siguiente manera:

\begin{equation*}
{bloque_n} = techo(\frac{playlist_{posicion}}{num\: playlists_{fichero}})  
\end{equation*}

A continuación, vamos a crear un diccionario donde almacenaremos a qué bloque pertenece cada conjunto. Este diccionario tendrá como clave el _id_ de la playlist y su valor será el bloque donde será almacenada:

In [7]:
location_tl_dict = dict()

with tqdm_nb(total=len(df_mpd_info)) as pbar:
    for pid, pl in df_mpd_info.iterrows():
        location_tl_dict[pl['id']] = math.floor(pid/NUM_LISTS_FILE)
        pbar.update()

___Ejemplo___:

Si tomamos como referencia la lista cuyo _id_ es `0GtTxbghpueJjaRxS6GOqp`:

In [8]:
df_mpd_info[df_mpd_info.id == '0GtTxbghpueJjaRxS6GOqp']

,id,collaborative,name,num_tracks,num_followers,num_artists,num_albums,duration_ms,modified_at,num_edits,file_name
652386,0GtTxbghpueJjaRxS6GOqp,False,Love Songs 1970's-1990's,204,2,158,189,48753425,1552521803,24,pls-set_06216.zip


<br>

Vemos que está en la posición _652386_ del DataFrame `df_mpd_info`. Con este dato, deducimos que esta lista pertenece al bloque que recoge las playlists que hay entre las posiciones 652000 y 652999. Con lo que el número del bloque al que pertenece es al _652_.

Si ahora vamos al diccionario y vemos el valor que contiene la clave `0GtTxbghpueJjaRxS6GOqp`:

In [9]:
location_tl_dict['0GtTxbghpueJjaRxS6GOqp']

652

Comprobamos que efectivamente se corresponde con el bloque al que pertenece dicha lista.

<br>

Antes de comenzar a generar nuestro DataSet, nos faltaría reunir las listas de pistas que pertenecen a cada playlist. Como estas listas están dispersas entre más de 10.000 ficheros, puesto que cuando descargamos las playlists se guardaban respecto al fichero de identificadores que la contenían, vamos a agruparlas según al bloque al que pertenece su playlist (diccionario `location_tl_dict`).

In [10]:
# Función mediante la cual se almacena un conjunto de listas de
# reproducción en base al bloque al que pertenecen dentro de nuestro
# DataSet
def store_tls_dump(set_id,tls,tls_sets_path):
    """
    :param set_id: Número de bloque al que pertenecen las listas a almacenar.
    :param tls: Conjunto de listas de pistas que queremos almacenar.
    :param tls_sets_path: Directorio donde almacenaremos los conjuntos.
    """
    # Variable que emplearemos para identificar el volcado
    dump_num = 0
    
    # Contamos los ficheros existentes para los volcados de datos del conjunto 
    # que vamos a almacenar. Lo emplearemos para el identificador del volcado del bloque
    for file_name in os.listdir(tls_sets_path):
        if file_name.startswith('track-lists-set_{:05d}.'.format(set_id)):
            dump_num += 1
    
    file_name = 'track-lists-set_{:05d}.dump{}.json'.format(set_id,dump_num)
    file_path = os.path.join(tls_sets_path,file_name)
       
    # Almacenamos el nuevo conjunto de listas
    with open(file_path, 'w', encoding='utf-8') as json_file:
        json.dump(tls, json_file, indent=4)

Cada vez que realizamos un volcado de listas de reproducción con un mismo identificador, estos datos se almacenan en un nuevo fichero. 

Por ejemplo, si realizamos 5 volcados de datos pertenecientes al bloque *598*, tendremos 5 ficheros diferentes. 

A estos volcados de datos, aparte de indicarse a qué bloque pertenecen, se les añadirá un sufijo *.dump_NUMERO-VOLCADO* para identificar dicho volcado.

Una vez que se termine el proceso de agrupación de las listas de pistas que pertenecen a un bloque, mediante la función ***merge_dump_sets*** nos encargaremos de unir en un sólo fichero todos aquellos volcados que pertenezcan a un bloque.

Este método funciona de la siguiente manera:

1. Omitiendo los sufijos *'.dump_'*, identificamos a qué bloques pertenecen los distintos volcados.
2. Mediante los nombres que hemos obtenido al eliminar los sufijos y que hemos almacenado en un conjunto, procedemos a identificar los volcados que pertenecen a los diferentes bloques que tenemos.
3. Leemos todos los volcados pertenecientes a un bloque.
4. Unimos los volcados en un único fichero.
5. Eliminamos los volcados tras crear el fichero único.

***Ejemplo***:

Los ficheros:
- *track-lists-set_109.dump0.json*
- *track-lists-set_109.dump1.json*
- *track-lists-set_109.dump2.json*
- *track-lists-set_109.dump3.json*
- *track-lists-set_109.dump4.json*

Pertenecen al bloque _109_. Tras leerlos, generamos un nuevo fichero y borramos los anteriores (que contenían la terminación *.dump_*):
- *track-lists-set_109.json*

In [11]:
# Método empleado para unir los volcados de datos pertenecientes a un bloque
def merge_dump_sets(tls_sets_path):
    """
    :param tls_sets_path: Directorio donde se encuentran los conjuntos de listas.
    """
    # Identificamos los bloques
    names_set = set()    
    for file in os.listdir(tls_sets_path):
        names_set.add(file.split('.')[0])
        
    # Buscamos los volcados que pertenecen a un mismo bloque
    # y los unimos en un único fichero
    pbar_merge = tqdm_nb(names_set)
    pbar_merge.set_description('Unión de ficheros')
    for name in pbar_merge:
        # Lista que contendrá todas las listas de pistas
        # pertenecientes a un bloque
        dump_tls_set = []
        
        # Lista que contiene los nombres de los ficheros
        # de los volcados de un bloque para posteriormente
        # borrarlos
        files_to_delete = []
        
        # Proceso de unión de listas de pistas
        for file in os.listdir(tls_sets_path):
            if file.startswith(name):
                file_path = os.path.join(tls_sets_path, file)
                files_to_delete.append(file_path)
                
                with open(file_path, 'r', encoding='utf-8') as json_file: 
                    data = json.load(json_file)                
                dump_tls_set += data

        # Guardamos un fichero con la unión de todos los volcados
        dump_file_name = f'{name}.json'
        dump_file_path = os.path.join(tls_sets_path, dump_file_name)
        with open(dump_file_path, 'w', encoding='utf-8') as json_file:
            json.dump(dump_tls_set, json_file, indent=4)
        
        # Una vez se han unido los volcados, procedemos a borrarlos
        for file_path in files_to_delete:
            os.remove(file_path)

<br>

Definimos también un método que emplearemos para comprimir los ficheros _JSON_ de un directorio:

In [12]:
# Comprime los ficheros JSON (uno a uno) de un directorio dado
def zip_json_files(folder_path):
    """
    :param folder_path: Directorio donde se encuentran ficheros JSON a comprimir.
    """
    # Comprimimos cada fichero.
    # Una vez comprimido eliminamos el fichero JSON
    for file in tqdm_nb(os.listdir(folder_path)):
        if file.endswith(".json"):
            file_path = os.path.join(folder_path, file)
            zip_file_name = os.path.splitext(file)[0] + ".zip"
            zip_file_path = os.path.join(folder_path, zip_file_name)
            with zipfile.ZipFile(zip_file_path,'w') as zip_file: 
                 zip_file.write(file_path, compress_type=zipfile.ZIP_DEFLATED, arcname=file)
            os.remove(file_path)

<br>

Comenzamos el proceso de generar los nuevos ficheros que contendrán las listas de pistas que deseamos obtener para nuestro DataSet.

<br>

<div class="alert alert-block alert-info">
    
<i class="fa fa-info-circle" aria-hidden="true"></i> __Nota__:
Para reducir el número de operaciones de lectura y escritura de los ficheros _JSON_ donde vamos a almacenar las listas, hemos establecido que estos datos se guardaran en los nuevos ficheros una vez se hayan leído 1.250 ficheros `.zip`.
</div>

<br>

<div class="alert alert-danger">
    
<i class="fa fa-info-circle" aria-hidden="true"></i> __Advertencia__:
El número de esta condición que acabamos de establecer, debe decidirse dependiendo del tamaño de la memoria RAM del equipo donde nos encontremos trabajando, puesto que podría darse el caso de que nos quedemos sin memoria disponible para la lectura de las listas de pistas.

</div>

<br>

In [13]:
def create_tls_sets(tls_path, pls_path, max_files):
    """
    :param tls_path: Directorio que contiene las listas de pistas.
    :param pls_path: Directorio que contiene las playlists obtenidas.
    :param max_files: Número máximo de listas de pistas que contiene un fichero.
    """        
    # Almacenamos los ids de aquellas playlist que tenemos en nuestro conjunto
    ids_set = set(df_mpd_info.id.to_list())

    # Diccionario que emplearemos para almacenar temporalmente
    # las listas de pistas que leemos de los ficheros comprimidos
    tl_set_dict = defaultdict(list)

    # Contador para almacenar el número de ficheros .zip que llevamos leídos.
    count = 0

    # Si no existe el directorio donde almacenaremos los
    # conjuntos de listas de pistas, lo creamos.
    if not os.path.exists(tls_path):
        os.mkdir(tls_path)
        
    last_file_name = os.listdir(pls_path)[-1]

    # Barra de progreso para el almacenamiento de las listas de pistas
    pbar_tls = tqdm_nb(total=NUM_LISTS_FILE, position=1)
    pbar_tls.set_description(f'Almacenamiento de datos')
    
    # Leemos cada fichero comprimido que se encuentre en el directorio 
    # donde están almacenadas las playlists y cargamos los datos que contiene
    pbar_read = tqdm_nb(os.listdir(pls_path))
    pbar_read.set_description('Lectura de ficheros')
    for file_name in pbar_read:
        file_path = os.path.join(pls_path,file_name)
        with zipfile.ZipFile(file_path,'r') as zip_file:
            with zip_file.open(zip_file.namelist()[0]) as json_file:
                data = json_file.read()
                list_of_pls = json.loads(data.decode())

        # Para cada playlist que hemos leído del fichero, comprobamos que están
        # en el conjunto de ids que aquellas listas que nos interesan y 
        # almacenamos la lista de pistas correspondiente
        for pl in list_of_pls:
            if pl['id'] in ids_set:
                tl_set_dict[location_tl_dict[pl['id']]].append({'id' : pl['id'], 'tracks' : pl['tracks']})
                ids_set.remove(pl['id'])

        # Aumentamos el contador de ficheros leídos
        count += 1

        # Comprobamos que hemos leído el número de ficheros establecido, o que 
        # es el último fichero que se va a tratar, para guardar los datos
        # en el nuevo conjunto de ficheros .json
        if (count == max_files) or (file_name == last_file_name):
            # Reiniciamos la barra de progreso del proceso de escritura
            pbar_tls.reset()
            
            for set_id, tls in tl_set_dict.items():
                store_tls_dump(set_id,tls,tls_path)
                pbar_tls.update()
            
            # Borramos el contenido de la lista temporal
            tl_set_dict = defaultdict(list)
            
            # Reiniciamos el contador
            count = 0
            
    merge_dump_sets(tls_path)

In [14]:
create_tls_sets(TLS_SETS_PATH,PLS_SETS_PATH,1250)

Como el tamaño de los nuevos ficheros que hemos generado para los conjuntos de listas de pistas de nuestras playlist es muy elevado, vamos a comprimirlos:

In [15]:
zip_json_files(TLS_SETS_PATH)

<br>

Una vez terminado el proceso anterior, procedemos a crear nuestro DataSet en formato *JSON*.

<div style="text-align: right">
<a href="#indice"><font size=5><i class="fa fa-arrow-circle-up" aria-hidden="true" style="color:#92002A"></i></font></a>
</div>

---

<a id="section3"></a>
## <font color="#92002A">3 - Generación del DataSet</font>
<br>

En este apartado, crearemos nuestro DataSet en formato [*JSON*](https://www.json.org/). El proceso que vamos a seguir, para cada bloque/fichero que debemos crear, es el siguiente:

1. Establecemos el valor '*info*' del fichero *JSON* (cabecera con los datos pertenecientes al bloque).
2. Leemos el fichero que contiene las listas de pistas de las playlist pertenecientes al bloque.
3. Para cada _id_ de la lista de reproducción que pertenece al bloque, cargamos sus datos junto a la lista de pistas y los guardamos en el valor '*playlist*' del fichero *JSON* a crear.
4. Generamos el fichero para posteriormente comprimirlo y quedarnos únicamente con el fichero `.zip`.

<div class="alert alert-block alert-info">
    
<i class="fa fa-info-circle" aria-hidden="true"></i> __Nota__:
Vamos a aprovechar el valor de la columna _index_ del DataFrame `df_mpd_info` para crear una nueva variable llamada ___pid___, que nos servirá para identificar una playlist dentro de nuestro conjunto (no confundir con *id*, que es el identificador de la lista en *Spotify*).
</div>

In [16]:
def create_mpd_json(mpd_path, tls_sets_path):
    """
    :param mpd_path: Directorio donde se almacenará el dataset en formato JSON.
    :param tls_sets_path: Directorio que contiene las playlists obtenidas.
    """    
    # Si no existe el directorio donde vamos a almacenar
    # los ficheros pertenecientes a nuestro DataSet, lo creamos.
    if not os.path.exists(mpd_path):
        os.makedirs(mpd_path)
        
    # Establecemos el número de elementos que contendrá cada fichero.
    chunk_size = NUM_LISTS_FILE

    # Contador donde indicamos el bloque actual.
    chunk_num = 0

    # Número de ficheros a generar.
    chunk_total = int(len(df_mpd_info) / chunk_size)

    # Creamos el esqueleto de la cabecera de cada bloque
    # (clave 'info' del diccionario).
    dic_info = dict()
    
    dic_info = {'generated_on' : str(datetime.datetime.now()),
                'slice' : '', 'version': 'v1.1'}

    # Creamos la barra para seguir el progreso de escritura 
    # de cada fichero.
    pbar_file = tqdm_nb(total=chunk_size, position=1)

    with tqdm_nb(total=chunk_total, position=0) as pbar_general:
        pbar_general.set_description('MPD')
        while chunk_num < chunk_total:
            pbar_file.reset()
            pbar_file.set_description(f'Slice {chunk_num + 1}')

            # Establecemos el conjunto de filas del DataFrame que
            # vamos a guardar en el bloque.
            start_pid = chunk_num * chunk_size
            end_pid = (chunk_num + 1) * chunk_size

            # Creamos el diccionario correspondiente al fichero
            # JSON que vamos a guardar y establecemos la cabecera.
            file = dict()        
            file['info'] = dic_info
            file['info']['slice'] = "{:06d}-{:06d}".format(start_pid, end_pid - 1)
            file['playlists'] = list()        
            file['playlists'] = []

            # Leemos el conjunto de listas de pistas que pertenecen 
            # a las playlists del conjunto.
            tls_file_name = 'track-lists-set_{:05d}.zip'.format(chunk_num)
            tls_file_path = os.path.join(tls_sets_path,tls_file_name)
            
            with zipfile.ZipFile(tls_file_path,'r') as zip_file:
                with zip_file.open(zip_file.namelist()[0]) as json_file:
                    data = json_file.read()
            list_of_tls = json.loads(data.decode())

            # Guardamos el contenido completo de cada playlist en 
            # un diccionario
            for pid, pl in df_mpd_info[start_pid:end_pid].iterrows():
                pl_dict_item = dict()

                pl_dict_item['pid'] = pid
                pl_dict_item['name'] = pl['name']
                pl_dict_item['collaborative'] = pl['collaborative']
                pl_dict_item['modified_at'] = pl['modified_at']
                pl_dict_item['num_albums'] = pl['num_albums']
                pl_dict_item['num_tracks'] = pl['num_tracks']
                pl_dict_item['num_followers'] = pl['num_followers']
                pl_dict_item['num_edits'] = pl['num_edits']
                pl_dict_item['duration_ms'] = pl['duration_ms']
                pl_dict_item['num_artists'] = pl['num_artists']
                pl_dict_item['tracks'] = next(tl['tracks'] for tl in list_of_tls if tl['id'] == pl['id'])
                for track in pl_dict_item['tracks']: del track['added_at']

                file['playlists'].append(pl_dict_item)
                pbar_file.update()

            # Creamos el fichero .json
            file_name = "mpd.slice.{:06d}-{:06d}.json".format(int(start_pid),int(end_pid - 1))
            file_path = os.path.join(mpd_path, file_name)
            with open(file_path, 'w') as f:
                json.dump(file,f,indent=4)

            # Comprimimos el fichero que hemos creado
            zip_file_name = file_name.replace('.json', '.zip')
            zip_file_path = os.path.join(mpd_path, zip_file_name)
            with zipfile.ZipFile(zip_file_path,'w') as zip_file: 
                 zip_file.write(file_path, compress_type=zipfile.ZIP_DEFLATED, arcname=file_name)

            # Eliminamos el fichero creado para quedarnos con
            # la versión comprimida
            os.remove(file_path)
            
            # Incrementamos el número de bloque
            chunk_num +=1
            pbar_general.update()        

In [17]:
create_mpd_json(MPD_PATH, TLS_SETS_PATH)

<br>

Una vez creado el DataSet en formato *JSON*, por seguridad, vamos a comprobar que el número de pistas almacenado en la variable '*num_tracks*' coincide con el tamaño de la lista de pistas:

In [18]:
def check_num_tracks(mpd_path):
    '''
    :param mpd_path: Directorio donde se encuentra el dataset en formato JSON.
    :return: Devuelve True/False si el 'num_tracks' coincide con el tamaño 
             de las listas de pistas
    '''
    # Variable que emplearemos para saber si en todas
    # las playlist coincide el valor de 'num_tracks'
    # con el tamaño de la lista de pistas ('tracks')
    same_values = True

    for file in tqdm_nb(os.listdir(mpd_path)):
        zip_file_path = os.path.join(mpd_path, file)
        with zipfile.ZipFile(zip_file_path,'r') as zip_file:
            with zip_file.open(zip_file.namelist()[0]) as json_file:
                data = json_file.read()
                slice_data = json.loads(data.decode())

        playlists = slice_data['playlists']

        for pl in playlists:
            if pl['num_tracks'] != len(pl['tracks']):
                same_values = False
                break

        if not same_values:
            break
            
    return same_values

In [19]:
check_num_tracks(MPD_PATH)

True

<br>

Vemos que la variable *num_track* coincide con el tamaño de la lista de pistas (*tracks*) para todas las playlists del conjunto.

Para finalizar, vamos a eliminar del directorio `DATA_PATH` aquellos ficheros que ya no nos resultarán de utilidad:

In [20]:
os.remove(os.path.join(DATA_PATH, 'downloaded_pls.csv'))
os.remove(os.path.join(DATA_PATH, 'downloaded_pls_artists.csv'))

<div style="text-align: right">
<a href="#indice"><font size=5><i class="fa fa-arrow-circle-up" aria-hidden="true" style="color:#92002A"></i></font></a>
</div>

---

<div style="text-align: right"> <font size=6><i class="fa fa-graduation-cap" aria-hidden="true" style="color:#92002A"></i> </font></div>